In [69]:
def LoadPackages():
    # Load Core Packages:
    global pd, time
    import pandas as pd 
    import time

    # Load Selenium Packages:
    global webdriver, By, WebDriverWait, Select, EC, TimeoutException, Keys
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait, Select
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.common.keys import Keys

    # Load Natural Language Processing Packages:
    global re, word_tokenize, sent_tokenize, regexp_tokenize, TweetTokenizer
    import re
    from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize, TweetTokenizer


def AssignSession():
    global command_url, session_id
    command_url = f'driver.command_executor._url: {driver.command_executor._url}'.split( )[1]
    session_id = f'driver.session_id: {driver.session_id}'

def WebDriver():
    global driver
    driver = webdriver.Chrome(r'C:\Users\Ahicks88\Documents\Briefcase\Analytics\GitHub\Tools-References\Tools\chromedriver')

def WebDriverContinue():
    global driver2
    driver2 = webdriver.Remote(command_executor=command_url)  
    if driver2.session_id != session_id:
        driver2.close()
        driver2.quit()   
    driver2.session_id = session_id

In [7]:
LoadPackages()
WebDriver()
driver.get('https://www.amazon.com/PlayStation-5-Console/product-reviews/B08FC5L3RG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
AssignSession()

In [8]:
WebDriverContinue()

reviews = []

# Skipped first 10 reviews because the format of the HTML alternates from [1] to [2] once you get to the second page for the review format variable.
driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a').click()
time.sleep(1)  

# Loops through all pages available for reviews of product
while True:
    j = 1
    # Loops through all 10 reviews available per page and stores it into the reviews list
    for j in range (2,12):
        try:
            review = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[4]/span/span'.format(j)).text
            reviews.append(review)
            j += 1
        except:
            pass
    try:    
        driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a').click()
        time.sleep(0.5) 
    except:
        break
    
print('{} reviews were added to the dataset.'.format(len(reviews)))

In [64]:
# Look at nth review in the dataset:
# reviews[0]

# Convert list into a single string
soup = "".join(reviews) 

# Look at head end of soup
# print(soup[:100])

# Look at tail end of soup
# print(soup[-100:])

In [113]:
tokenized_soup = word_tokenize(soup)
summary_reviews = len(reviews);                     print('There are {} reviews in the dataset'.format(summary_reviews))
summary_sentences = len(sent_tokenize(soup));       print('There are {} sentences in the dataset'.format(summary_sentences))
summary_words = len(tokenized_soup);                print('There are {} words in the soup'.format(summary_words))

KPI_Avg_sent_review = round(summary_sentances/summary_reviews,2);       print('There is an average of {} sentences per review'.format(KPI_Avg_sent_review))
KPI_Avge_word_review = round(summary_words/summary_reviews,2);          print('There is an average of {} words per review'.format(KPI_Avge_word_review))

There are 2027 reviews in the dataset
There are 4988 sentences in the dataset
There are 101287 words in the soup
There is an average of 2.46 sentences per review
There is an average of 49.97 words per review


2.46

In [87]:
reviews

ony really outdid themselves for the new generation.\nAlso a huge bonus that it still plays all of the now old ps4 games.\nDefinitely a Christmas showstopper!',
 'Perfection! Plus you get that 2-day fast delivery with Amazon Prime!',
 'Super dope system that makes all the PS4 games beautiful and the ps5 games look breath taking',
 'This console is amazing. Plays my PS4 exclusives flawless and ps5 game are really fun with the new control. I am enjoying this console a lot.',
 'Finally got my ps5. Thanks Amazon product works great.',
 'This console is amazing! The speed is so much faster, best frame rate, better graphics. I absolutely love it!',
 'Controller is truly next gen, not much to say. Big upgrade over my ps4 slim and competitive with my PC performance too (for now)',
 'Amazing Console. Love it!',
 'Because it is the same as the PS4 it was a lot of money $500 no new games this PS five is garbage',
 'I love it and it came alot quicker than I thought. Double the price but worth it i